# ДЗ 3 
## Реализация BM25

## Функция ранжирования BM25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Query$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $Doc$ запросу $Query$:

$$ BM25(Query, Doc) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,Doc)*(k+1)}{TF(q_i,Doc)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где    
$$$$
$\text{IDF}(q_i)$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в корпусе   
$n(q_i)$ — количество документов, содержащих слово $q_i$

>$TF(q_i,Doc)$ - частота слова $q_i$ в документе $Doc$    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75  
$l(d)$ - длина документа (количество слов в нём)   
$avgdl$ — средняя длина документа в корпусе    

### __Задача__:

Реализуйте поиск, где
- в качестве векторизации документов корпуса - слагаемые **BM25**
- формат хранения индекса - **матрица Document-Term**
- метрика близости пар (запрос, документ) - **BM25**

В реализации должно быть все то же, что во втором дз:
- функция индексации корпуса, на выходе которой посчитанная матрица Document-Term
- функция индексации запроса, на выходе которой посчитанный вектор запроса
- функция с реализацией подсчета близости запроса и документов корпуса, на выходе которой вектор, i-й элемент которого обозначает близость запроса с i-м документом корпуса. Сделать **обязательно векторно**.
- главная функция, объединяющая все это вместе; на входе - запрос, на выходе - отсортированные по убыванию имена документов коллекции

Обратите внимание:
- сортировку надо сделать **<font color='green'>обязательно векторно</font>** через маску **(ниже дан пример)**; при несоблюдении минус два балла
- подсчет индекса надо сделать **<font color='green'>обязательно с использованием спарс-матриц</font>**, то есть ни в какой момент времени векторизованный корпус не переводится в ndarray; при несоблюдении минус балл


В качестве корпуса возьмите корпус вопросов и ответов с Ответы Мейл) 👍😃 
[Ссылка для скачивания](https://www.kaggle.com/bobazooba/thousands-of-questions-about-love)

Описание структуры данных можно посмотреть по ссылке. В качестве документов корпуса берем значения из ключа *answers*, но не все, а один, у которого максимальный *value*. При этом ограничиваем количество документов до 50000. Пример ниже.


**На что направлена эта задача:** 
Реализация поисковика с использованием BM25.



In [ ]:
# # датасет для индексирования

import json

with open('questions_about_love.jsonl', 'r') as f:
    corpus = list(f)[:50000]

# пример элемента оригинального датасета 
json.loads(corpus[22])

{'question': 'Кто что должен сделать!? Для завоевания доверия женщины у мужчины и мужчины у женщины, для прочных отношений!',
 'comment': '',
 'sub_category': 'relations',
 'author': 'Diesel',
 'author_rating': {'category': 'Мыслитель', 'value': '5175'},
 'answers': [{'text': 'это подозрительно когда настойчиво навязывают чувство доверия',
   'author_rating': {'category': 'Знаток', 'value': '312'}},
  {'text': 'Пересказ информации вайшнавов. Доверие складывается из 2 штук. 1. Доброта - пилот добрый, но не умеет водить самолет - лететь страшно, доверия нет. 2. Профессионализм - зирург отличный, но садист, отрежет лишнее - нет доверия.Итак, учитывайте потребности человека, повышайте айкью, чтоб по внешнему виду определять че человеку надо, не плюйте на его состояние (не просите больного гриппом идти для вас в аптеку за презиками), покажите, что вы никогда не будете его насиловать - в случае если вас что-то не устроит просто уйдете. Говорите правду. Желательно не косячить - такую правду г

In [ ]:
# пример векторной сортировки

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


corpus = [
    'мыла',
    'раму',
    'киса',
    'мама мыла раму',
    'мама мыла раму',
    'мама мыла раму'
]
corpus_doc_names = np.array(['name_1', 'name_2', 'name_3', 'name_4', 'name_5', 'name_6'])
query = 'мама мыла раму'


vectorizer = TfidfVectorizer()
corpus_matrix = vectorizer.fit_transform(corpus)
query_vec = vectorizer.transform([query]).toarray()

# считаем косинусную близость
scores = cosine_similarity(corpus_matrix, query_vec)

# сортируем индексы скоров в обратном порядке (по убыванию)
sorted_scores_indx = np.argsort(scores, axis=0)[::-1]

# сортируем имена файлов в соответствии со скорами
corpus_doc_names[sorted_scores_indx.ravel()]

array(['name_6', 'name_5', 'name_4', 'name_2', 'name_1', 'name_3'],
      dtype='<U6')

In [ ]:
# подсказки про спарс-матрицу

from scipy import sparse


# итерация по ненулевым элементам спарс-матрицы
# for i, j in zip(*sparce_matrix.nonzero()): 
#     ...
    
# создать спарс-матрицу из данных, где
# values - лист из n значений, которые мы хотим положить в матрицу 
# rows - лист из n значений, где i-тое значение это индекс строки i-го элемента из values
# cols - лист из n значений, где i-тое значение это индекс колонки i-го элемента из values

values = [99, 22, 77]
rows = [0, 2, 3]
cols = [0, 2, 4]


sparse.csr_matrix((values, (rows, cols)))

<4x5 sparse matrix of type '<class 'numpy.longlong'>'
	with 3 stored elements in Compressed Sparse Row format>

## Imports

In [2]:
! pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 8.2 MB 7.8 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=523080fa4dcf041f6aeb769f7ad774da900e61d8a512f0bef58b168f07d5ee3e
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import nltk
import json
import numpy as np
from pymorphy2 import MorphAnalyzer
from nltk.tokenize import WordPunctTokenizer
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from scipy import sparse

In [5]:
morph = MorphAnalyzer()
tokenizer = WordPunctTokenizer()
nltk.download("stopwords")
stop_words = set(stopwords.words("russian"))
count_vectorizer = CountVectorizer()
tf_vectorizer = TfidfVectorizer(use_idf=False, norm='l2')
tfidf_vectorizer = TfidfVectorizer(use_idf=True, norm='l2')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Processing

preprocess texts:

In [6]:
def preprocess(text):
  text = tokenizer.tokenize(text.lower())
  lemmas = list()
  for t in text:
    if t not in stop_words and t not in punctuation:
      lemmas.append(morph.parse(t)[0].normal_form)

  return ' '.join(lemmas)

gather relevant answers (those with maximum value) from corpus

In [7]:
def get_relevant(path):
  with open(path, 'r', encoding='utf-8') as f:
    corpus = list(f)[:50000]
  lemmas = list()
  texts = list()
  for i in corpus:
    answers = json.loads(i)['answers']
    if answers:
      answer_values = np.array(map(int, [i['author_rating']['value'] for i in answers if i != '']))
      answer = answers[np.argmax(answer_values)]['text']
      lemmas.append(preprocess(answer))
      texts.append(answer)
  return lemmas, texts

build matrix:

In [8]:
def get_indexes(corpus, k=2, b=0.75):
  x_count = count_vectorizer.fit_transform(corpus)
  x_idf = tfidf_vectorizer.fit_transform(corpus)
  x_tf = tf_vectorizer.fit_transform(corpus)
  idf = tfidf_vectorizer.idf_
  len_d = x_count.sum(axis=1)
  avdl = len_d.mean()
  fin = k * (1 - b + b * len_d / avdl)
  matrix = sparse.lil_matrix(x_tf.shape)

  for i, j in zip(*x_tf.nonzero()):
    matrix[i, j] = (x_tf[i, j] * (k + 1) * idf[j])/(x_tf[i, j] + fin[i])
    
  return matrix.tocsr()

In [9]:
def find(query, corpus, answers):
  lemmas = preprocess(query)
  if lemmas:
    query_index = count_vectorizer.transform([lemmas])
    bm25 = corpus.dot(query_index.T)
    i = np.argsort(bm25.toarray(), axis=0)
    return np.array(answers)[i][::-1].squeeze()
  else:
    pass

In [ ]:
path = '/content/drive/MyDrive/infosearch22/project/data.jsonl'
corpus, lemmas = get_relevant(path)
matrix = get_indexes(lemmas)
qr = input('You may type in your query or "STOP" if you want to stop: ')
check = True
while check == True:
  qr = input('You may type in your query or "STOP" if you want to stop: ')
  if 'STOP' not in qr:
    result = find(qr, matrix, corpus)
    print(*result[:20])
  else:
    check = False